In [1]:
import os
import h5py
import cv2
import numpy as np
import pandas as pd
import sklearn as skl
from matplotlib import pyplot as plt

from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

from OCRPage import OCRPage

In [9]:
# set home directory
home_path = os.getcwd()[:os.getcwd().find('Code/python/obf_autax/ocr')-1]
# set input_path
input_path = os.path.join(home_path, 'Data', 'obf_autman', 'pdf', '171_12_PS_Pernitz.pdf')
#Operat Hinterriß
#'171_11_GS_Mannersdorf.pdf'

In [10]:
def convert_to_image(input_path, page_nr):

    pages = convert_from_path(input_path, 500, first_page=page_nr, last_page=page_nr)

    return pages[0]

In [11]:
#img = convert_to_image(input_path, 3).convert('RGB')
#img = np.array(img)
# 113

img = convert_to_image(input_path, 3).convert('RGB')
img = np.array(img)
ocr = OCRPage(img)
ocr.convert()

## Text

In [12]:
# get text
txt = ocr.get_text('text')

In [33]:
# split lines
txt = txt.splitlines()

In [34]:
txt

['ST steilerer Abfall zu "BO", am Rücken blockig, BE einzelne SK, BU im NordOsten, ungleichaltrig +/- 20 Jahre, geringe',
 'Qualität, meist krummwüchsig, mäßig vergrast und verkrautet']

In [16]:
txt[1].replace('El', 'EI')
txt[1].replace('Kl', 'KI')
txt[1].replace('Zl', 'ZI')

'Qualität, meist krummwüchsig, mäßig vergrast und verkrautet'

In [17]:
txt

['ST steilerer Abfall zu "BO", am Rücken blockig, BE einzelne SK, BU im NordOsten, ungleichaltrig +/- 20 Jahre, geringe',
 'Qualität, meist krummwüchsig, mäßig vergrast und verkrautet']

In [18]:
len(txt)

2

In [61]:
for i in range(3-2):
    txt.append("")

In [62]:
l0, l1, l2 = txt

In [16]:
img = convert_to_image(input_path, 150).convert('RGB')
img = np.array(img)
ocr = OCRPage(img)
ocr.convert()

## Maßnahme

In [17]:
ma = ocr.get_text('mass')
spaces = ocr.get_spaces()

In [27]:
dic_ma = {'S':0, 'MA':1, 'FL':0, 'LH':1, 'NH':1, 'SUM':0, 'DR':1, 'BH':1, 'ZP':1, 'SG':1, 'RU':1}

In [28]:
ma

'DF090213435\nEnde Dezennium bzw Anfang nächstes Dezennium'

In [29]:
# 1
def split_lines(ma, dic_ma):
    ma = ma.splitlines()
    row_info = []
    for i in range(len(ma)):
        if ma[i][dic_ma['S']:dic_ma['S']+2] in ["DE", "DF", "RM", "LI", "AD", "KH"]:
            ma[i] = ma[i].replace(" ", "")
            row_info.append("nutz")
        else:
            ma[i] = ma[i].replace("Bl", "BI")
            ma[i] = ma[i].replace("El", "EI")
            ma[i] = ma[i].replace("Kl", "KI")
            ma[i] = ma[i].replace("Ll", "LI")
            ma[i] = ma[i].replace("Zl", "ZI")
            row_info.append("text")
        
    return ma, row_info

In [30]:
data, info = split_lines(ma, dic_ma)

print(data, info)

rec_fil,rows_pos = get_char_positions(ocr.img_thresh)

print(rows_pos)

distances = get_dist_char(data, dic_ma, rec_fil, rows_pos, info)

print(distances)

print(data[0])
print(dic_ma)
z = from_string(data[0], dic_ma, distances[0])
z

['DF090213435', 'Ende Dezennium bzw Anfang nächstes Dezennium'] ['nutz', 'text']
[250 252 346 347 348 364]


ValueError: cannot convert float NaN to integer

In [31]:
rec_fil

array([[1943,  346,    8,   43],
       [ 687,  346,    8,   43],
       [1961,  347,   35,   45],
       [1759,  346,   37,   44],
       [1226,  364,   39,   62],
       [ 929,  346,   57,   44],
       [ 888,  346,   37,   44],
       [ 706,  347,   35,   45],
       [ 504,  346,   37,   44],
       [2008,  346,   59,   45],
       [1896,  346,   36,   45],
       [1850,  346,   36,   45],
       [1802,  348,   40,   47],
       [1714,  348,   40,   47],
       [1589,  348,   36,   47],
       [1542,  348,   40,   47],
       [1477,  348,   36,   47],
       [1388,  347,   38,   46],
       [1343,  347,   42,   46],
       [1299,  346,   36,   45],
       [1183,  346,   36,   45],
       [1135,  347,   42,   46],
       [1067,  346,   36,   45],
       [ 752,  346,   59,   45],
       [ 641,  346,   36,   45],
       [ 595,  346,   36,   45],
       [ 546,  348,   40,   47],
       [ 458,  348,   40,   47],
       [ 329,  348,   40,   47],
       [ 238,  346,   36,   45],
       [15

In [172]:
distances[0]

{'dist': array([ 5, 17, 19,  7,  6, 20]), 'thresh': 18}

In [21]:
def from_string(ma_as_str, dic_ma, distances):
    # TYP
    # first 2 -> Maßnahme
    # last 6 -> Nutzcode
    
    dic_ = {'S':0, 'MA':0, 'FL':0, 'LH':0, 'NH':0, 'SUM':0, 'DR':0, 'BH':0, 'ZP':0, 'SG':0, 'RU':0}
    
    #len_tech = dic_ma['DR'] + dic_ma['BH'] + dic_ma['ZP'] + dic_ma['SG'] + dic_ma['RU']*2
    
    # set Schicht (S)
    if dic_ma['S']:
        dic_['S'] = ma_as_str[:dic_ma['S']]
        
    # set Maßnahme (MA)
    if dic_ma['MA']:
        dic_['MA'] = ma_as_str[dic_ma['S']:dic_ma['S']+2]
    
    pos = 0
    
    # set Rückung (RU)
    if dic_ma['RU']:
        pos = -2
        dic_['RU'] = ma_as_str[pos:]
        
    # set Schlägerung (SG), Zeitpunkt (ZP), Behörde (BH), Dringlichkeit (DR)
    for i in ['SG', 'ZP', 'BH', 'DR']:
        if dic_ma[i]:
            pos -= 1
            dic_[i] = ma_as_str[pos:pos+1]
            
    #print(ma_as_str[dic_ma['S']+2:pos])
    #print(distances["dist"])
    
    # pharse FL - LH - NH string
    data_ma = pharse_string(ma_as_str[dic_ma['S']+2:pos], distances["dist"], distances["thresh"])
    
    # set Fläche (FL), Laubholz (LH), Nadelholz (NH), Summe (SUM)
    cur = 0
    for typ in ['FL', 'LH', 'NH', 'SUM']:
        if dic_ma[typ]:
            dic_[typ] = data_ma[cur]
            cur += 1 
    
    return dic_

In [226]:
print(data[0])
print(dic_ma)
z = from_string(data[0], dic_ma, distances[0])
z

DF10700013435
{'S': 0, 'MA': 1, 'FL': 1, 'LH': 1, 'NH': 1, 'SUM': 0, 'DR': 1, 'BH': 0, 'ZP': 1, 'SG': 1, 'RU': 1}


{'S': 0,
 'MA': 'DF',
 'FL': '10',
 'LH': '700',
 'NH': '0',
 'SUM': 0,
 'DR': '1',
 'BH': 0,
 'ZP': '3',
 'SG': '4',
 'RU': '35'}

In [32]:
img_thresh = ocr.img_thresh
rec_fil,rows_pos = get_char_positions(img_thresh)

In [33]:
info

['nutz', 'text']

In [34]:
rec_fil

array([[1943,  346,    8,   43],
       [ 687,  346,    8,   43],
       [1961,  347,   35,   45],
       [1759,  346,   37,   44],
       [1226,  364,   39,   62],
       [ 929,  346,   57,   44],
       [ 888,  346,   37,   44],
       [ 706,  347,   35,   45],
       [ 504,  346,   37,   44],
       [2008,  346,   59,   45],
       [1896,  346,   36,   45],
       [1850,  346,   36,   45],
       [1802,  348,   40,   47],
       [1714,  348,   40,   47],
       [1589,  348,   36,   47],
       [1542,  348,   40,   47],
       [1477,  348,   36,   47],
       [1388,  347,   38,   46],
       [1343,  347,   42,   46],
       [1299,  346,   36,   45],
       [1183,  346,   36,   45],
       [1135,  347,   42,   46],
       [1067,  346,   36,   45],
       [ 752,  346,   59,   45],
       [ 641,  346,   36,   45],
       [ 595,  346,   36,   45],
       [ 546,  348,   40,   47],
       [ 458,  348,   40,   47],
       [ 329,  348,   40,   47],
       [ 238,  346,   36,   45],
       [15

In [35]:
distances = get_dist_char(data, dic_ma, rec_fil, rows_pos, info)

ValueError: cannot convert float NaN to integer

In [22]:
def get_dist_char(data, dic_ma, rec_fil, rows_pos, info):
    '''
    get distances between characters
    dic_ma = {'S':0, 'MA':1, 'FL':1, 'LH':1, 'NH':1, 'SUM':0, 'DR':1, 'BH':0, 'ZP':1, 'SG':1, 'RU':1}
    string = 'DF13,8700023435'
    '''
    distances = []
    ### get info about structure
    len_1 = dic_ma['S'] + dic_ma['MA']*2
    len_2 = dic_ma['DR'] + dic_ma['BH'] + dic_ma['ZP'] + dic_ma['SG'] + dic_ma['RU']*2
    
    for i, line in enumerate(data):
        # filter only nutzungs row
        if info[i] == 'nutz':
            rec = rec_fil[np.where(rec_fil[:,1] == rows_pos[i])]

            # sort
            rec.view('i8,i8,i8,i8').sort(order=['f1'], axis=0)

            # filter just relevant 
            rec = rec[len_1:-len_2]

            width_m = np.median(rec[:,2,])
            pos_x = rec[:,0,].copy()
            dist = pos_x[1:] - (pos_x[:-1]+int(width_m))

            thresh = otsu(dist)

            thisdict = {"dist": dist, "thresh": thresh}
            distances.append(thisdict)
            
    return distances

In [23]:
#
def otsu(gray):
    '''
        otsu algorithm for finding the best threshold value
    '''
    pixel_number = gray.shape[0]
    mean_weigth = 1.0/pixel_number
    his, bins = np.histogram(gray, np.array(range(0, np.max(gray))))
    final_thresh = -1
    final_value = -1
    for t in bins[1:-1]: # This goes from 1 to 254 uint8 range (Pretty sure wont be those values)
        Wb = np.sum(his[:t]) * mean_weigth
        Wf = np.sum(his[t:]) * mean_weigth

        mub = np.mean(his[:t])
        muf = np.mean(his[t:])

        value = Wb * Wf * (mub - muf) ** 2

        if value > final_value:
            final_thresh = t
            final_value = value

    return(final_thresh)

In [24]:
# OCRPage
def get_char_positions(img_thresh):
    # invert binary
    img = cv2.bitwise_not(img_thresh)
    # cut the mass
    img = ocr.cut_feature(img, "mass")
    # get boundries of detected characters
    rects = ocr.find_objects(img)
    # convert to numpy array
    rects = np.asarray(rects)
    
    # filter noise (small points)
    rec_fil = rects[rects[:,3] > np.mean(rects[:,3])/2]
    # set lower positions
    rec_fil[:,1] = rec_fil[:,1] + rec_fil[:,3]
    # get unique row positions
    rows_pos = np.sort(np.unique(rec_fil[:,1]))
    
    return(rec_fil,rows_pos)

In [25]:
def pharse_string(string, dist, thresh_val):
    '''
    pharse string into usable variables based on the distance between the characters
    input:  string = '13,87000'
            dist = np.array([ 5, 17, 19,  7,  6, 20])
    output: mass = ['13,8', '700', '0']
    '''
    # list of variables
    mass = []
    # if comma is detected swich to 1
    comma = 0
    # temporary helper variable
    save = string[0]
    # loop over char in string
    for i, char in enumerate(string[1:]):
        # if char is comma -> add char and reuse dist on ith position
        if char == ',':
            save = save + char
            dist[i] = thresh_val - 1
            comma = 1
        # check if current dist is bigger than threshold -> add to list of variables
        else:
            if dist[i-comma] <= thresh_val:
                save = save + char
            else:
                mass.append(save)
                save = char
    # add last variable
    mass.append(save)
    
    return mass

In [210]:
string = '13,87000'
dist = np.array([ 5, 17, 19,  7,  6, 20])
pharse_string(string, dist, 17)

['13,8', '700', '0']

In [213]:
string = '14,510000'
dist = np.array([ 2, 11, 10,  3,  4,  3, 11])
pharse_string(string, dist, 10)

['14,5', '1000', '0']

In [36]:
# invert binary
img = cv2.bitwise_not(img_thresh)
# cut the mass
img = ocr.cut_feature(img, "mass")
# get boundries of detected characters
rects = ocr.find_objects(img)
# convert to numpy array
rects = np.asarray(rects)

In [37]:
rects

array([[1943,  303,    8,   43],
       [ 687,  303,    8,   43],
       [1961,  302,   35,   45],
       [1759,  302,   37,   44],
       [1226,  302,   39,   62],
       [ 929,  302,   57,   44],
       [ 888,  302,   37,   44],
       [ 706,  302,   35,   45],
       [ 504,  302,   37,   44],
       [2008,  301,   59,   45],
       [1896,  301,   36,   45],
       [1850,  301,   36,   45],
       [1802,  301,   40,   47],
       [1714,  301,   40,   47],
       [1589,  301,   36,   47],
       [1542,  301,   40,   47],
       [1477,  301,   36,   47],
       [1388,  301,   38,   46],
       [1343,  301,   42,   46],
       [1299,  301,   36,   45],
       [1183,  301,   36,   45],
       [1135,  301,   42,   46],
       [1067,  301,   36,   45],
       [ 752,  301,   59,   45],
       [ 641,  301,   36,   45],
       [ 595,  301,   36,   45],
       [ 546,  301,   40,   47],
       [ 458,  301,   40,   47],
       [ 329,  301,   40,   47],
       [ 238,  301,   36,   45],
       [15

In [38]:
# filter noise (small points)
rec_fil = rects[rects[:,3] > np.mean(rects[:,3])/2]
# get amount of Nutzungsmaßnahmen
rec_fil[:,1] = rec_fil[:,1] + rec_fil[:,3]
# get unique row positions
rows_pos = np.sort(np.unique(rec_fil[:,1]))

In [39]:
rec_fil

array([[1943,  346,    8,   43],
       [ 687,  346,    8,   43],
       [1961,  347,   35,   45],
       [1759,  346,   37,   44],
       [1226,  364,   39,   62],
       [ 929,  346,   57,   44],
       [ 888,  346,   37,   44],
       [ 706,  347,   35,   45],
       [ 504,  346,   37,   44],
       [2008,  346,   59,   45],
       [1896,  346,   36,   45],
       [1850,  346,   36,   45],
       [1802,  348,   40,   47],
       [1714,  348,   40,   47],
       [1589,  348,   36,   47],
       [1542,  348,   40,   47],
       [1477,  348,   36,   47],
       [1388,  347,   38,   46],
       [1343,  347,   42,   46],
       [1299,  346,   36,   45],
       [1183,  346,   36,   45],
       [1135,  347,   42,   46],
       [1067,  346,   36,   45],
       [ 752,  346,   59,   45],
       [ 641,  346,   36,   45],
       [ 595,  346,   36,   45],
       [ 546,  348,   40,   47],
       [ 458,  348,   40,   47],
       [ 329,  348,   40,   47],
       [ 238,  346,   36,   45],
       [15

In [40]:
rows_pos

array([250, 252, 346, 347, 348, 364])

In [30]:
# filter only nutzungs row
rec_fil_fil = rec_fil[np.where(rec_fil[:,1] == rows_pos[0])]

In [45]:
rec_fil[-1][1] + rec_fil[-1][1] * 0.15

287.5

In [47]:
# filter only nutzungs row
rec_fil_fil = rec_fil[np.where((rec_fil[:,1] <= rec_fil[-1][1] + rec_fil[-1][1] * 0.15) & (rec_fil[:,1] >= rec_fil[-1][1] - rec_fil[-1][1] * 0.15))]

In [48]:
rec_fil_fil

array([[824, 252,  40,  59],
       [777, 252,  41,  60],
       [638, 252,  41,  60],
       [574, 250,  22,  58],
       [430, 252,  40,  60],
       [384, 252,  39,  60],
       [314, 252,  40,  60],
       [707, 250,  42,  59],
       [500, 250,  40,  59],
       [244, 250,  42,  60],
       [184, 250,  49,  60]])

In [32]:
# sort
a = rec_fil_fil.copy()
a.view('i8,i8,i8,i8').sort(order=['f1'], axis=0)

In [33]:
a

array([[ 648,  475,   28,   39],
       [ 685,  475,   24,   39],
       [ 729,  475,   15,   39],
       [ 759,  475,   24,   40],
       [ 801,  475,   25,   40],
       [ 845,  475,   27,   39],
       [ 877,  475,   25,   40],
       [ 908,  475,   25,   40],
       [ 953,  475,   25,   40],
       [ 998,  475,   26,   40],
       [1043,  475,   24,   40],
       [1086,  475,   29,   39],
       [1133,  475,   24,   40],
       [1165,  475,   25,   39]])

In [36]:
a2 = a[2:-5]

In [87]:
pos_x = a2[:,0,].copy()

In [88]:
pos_x

array([729, 759, 801, 845, 877, 908, 953])

In [48]:
a2

array([[729, 475,  15,  39],
       [759, 475,  24,  40],
       [801, 475,  25,  40],
       [845, 475,  27,  39],
       [877, 475,  25,  40],
       [908, 475,  25,  40],
       [953, 475,  25,  40]])

In [59]:
for i in range(a2.shape[0]-1):
    print(a2[i+1,0] - (a2[i,0] + int(width_m)))

5
17
19
7
6
20


In [57]:
width_m = np.median(a2[:,2,])

In [58]:
width_m

25.0

In [89]:
diff = pos_x[1:] - (pos_x[:-1]+int(width_m))

In [90]:
diff

array([ 5, 17, 19,  7,  6, 20])

In [ ]:
def get_spaces(img_thresh): # public
        '''
            get spaces between maßnahmen code
        '''
        # invert binary
        img = cv2.bitwise_not(self.img_thresh)
        # cut the mass
        img = self.cut_feature(img, "mass")
        # get boundries of detected characters
        rects = self.find_objects(img)
        # convert to numpy array
        rects = np.asarray(rects)

### TODO dont filter commas

        # filter noise (small points)
        rec_fil = rects[rects[:,3] > np.mean(rects[:,3])/2]
        # get amount of Nutzungsmaßnahmen
        rec_fil[:,1] = rec_fil[:,1] + rec_fil[:,3]
        # get unique row positions
        rows_pos = np.sort(np.unique(rec_fil[:,1]))

        spaces = []

        for i in range(0, rows_pos.size-1, 2):
            # filter only nutzungs row
            rec_fil_fil = rec_fil[np.where(rec_fil[:,1] == rows_pos[i])]
            # invert array on first column
            rec_fil_fil.view('i8,i8,i8,i8').sort(order=['f1'], axis=0)
            # position of char on x-axis -> column 0
            pos_x = rec_fil_fil[:,0,].copy()
            # median width > column 2
            width_m = np.median(rec_fil_fil[:,2,])
            # space (pixels) between characters
            diff = pos_x[1:] - (pos_x[:-1]+int(width_m))
            # use otsu algorithm to obtain the best threshold value
            thresh_val = self.otsu(diff)

            diff[np.where(diff < thresh_val)] = 0
            diff[np.where(diff >= thresh_val)] = 1

            spaces.append(diff)

### TODO add possibitity for 2 or more Maßnahmen and one without text

        ## search for commas
        # get Maßnahme
        mass = self.get_text("mass")
        mass = mass.splitlines()
    ### TODO
        # find commas
        idx_comma = mass[0].find(',')
        # change space to no space
        spaces[0][idx_comma-1] = 0
        # add no space, since comma is not recognised as char
        spaces[0] = np.insert(spaces[0],3,0)

        return(spaces)


In [12]:
def from_string(ma_as_str, spaces):
    # split lines
    ma = ma_as_str.splitlines()
    nutz = ma[0].replace(" ", "")
    l = []
    ch = ""

    for i in range(len(nutz)):

        if i == 0:
            ch = nutz[i]
        elif i == len(nutz)-1:
            l.append(ch + nutz[i])
        elif spaces[0][i-1] == 0:
            ch = ch + nutz[i]
        elif spaces[0][i-1] == 1:
            l.append(ch)
            ch = nutz[i]
    l.append(ma[1])
    return l

In [17]:
from_string(ma, spaces)

['DF13,8700', '0', '2', '34', '35', 'El fördern u freistellen']

In [47]:
spaces

[array([0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1])]

In [47]:
ma = 'DF4,7300023435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

In [49]:
ma = 'DF47300023435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '473', '000', '2', '3', '4', '35', 'El Kronen freistellen']

In [50]:
# added 1 to spaces and ma
ma = 'DF4,73000213435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '4,7', '300', '0', '2', '1', '3', '4', '35', 'El Kronen freistellen']

In [51]:
# added 1 to spaces and ma
ma = 'DF213435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 1, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '2', '1', '3', '4', '35', 'El Kronen freistellen']

In [9]:
# split lines
ma = ma.splitlines()

In [10]:
ma

['DF4,7300023435', 'El Kronen freistellen']

In [44]:
l = []
ch = ""

for i in range(len(ma[0])):
    
    if i == 0:
        ch = ma[0][i]
    elif i == len(ma[0])-1:
        l.append(ch + ma[0][i])
    elif spaces[0][i-1] == 0:
        ch = ch + ma[0][i]
    elif spaces[0][i-1] == 1:
        l.append(ch)
        ch = ma[0][i]
l.append(ma[1])

In [45]:
l

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

## BZ

In [10]:
bz = "3BU 4FI 3AH"

In [11]:
bz = bz.split()

In [12]:
bz

['3BU', '4FI', '3AH']

In [13]:
bz[0][-2:]

'BU'

In [14]:
int(bz[0][:-2])

3

In [18]:
bzs = []
for b in bz:
    bzs.append([int(b[:-2]),b[-2:]])

In [20]:
# Return double of n 
def addition(n): 
    return n + n 
  
# We double all numbers using map() 
numbers = (1, 2, 3, 4) 
result = map(addition, numbers) 
print(list(result))

[2, 4, 6, 8]


In [21]:
bz

['3BU', '4FI', '3AH']

In [27]:
def spliter(n):
    return int(n[:-2]),n[-2:]
result = map(spliter, bz)

In [28]:
result

In [19]:
bzs

[[3, 'BU'], [4, 'FI'], [3, 'AH']]

In [1]:
from OCRData import BestZiel
from OCRData import BAZiel

In [2]:
ba1 = BAZiel(13, 'BU')
ba2 = BAZiel(4, 'K')
ba3 = BAZiel(3, 'LA')

In [3]:
bz = BestZiel([ba1, ba2, ba3])

In [8]:
bz = BestZiel.from_string('3BI 4KI 3FI')

3BI
3
BI
4KI
4
KI
3FI
3
FI


In [9]:
bz

BestZiel(bz=[BAZiel(part=3, ba='BI', error=False), BAZiel(part=4, ba='KI', error=False), BAZiel(part=3, ba='FI', error=False)], error=False)

In [10]:
bz.check_attr()

In [11]:
bz

BestZiel(bz=[BAZiel(part=3, ba='BI', error=False), BAZiel(part=4, ba='KI', error=False), BAZiel(part=3, ba='FI', error=False)], error=False)

In [6]:
from OCRData import Text

In [9]:
# get text
txt = ocr.get_text('text')

In [10]:
Text.from_string(txt)

Text(line0='ST im Süden breiter Rücken, am Rücken steinig mit STOE 71 und HD-Typ', line1='BE einige ES KB BU BI AH Kl, Elam Rücken, einige eingewachsene BU-El-Überhälter, ungleichaltrig +/- 10 Jahre, Ausschlagswald', line2='')

In [24]:
from OCRData import Nutz

In [46]:
nutz = Nutz(ma=l[0], v_lh=int(l[2]), v_nh=int(l[3]), dring=int(l[4]), behoerde=0, zeitp=int(l[5]), \
            schlaeg=int(l[6]), rueck=int(l[7]), text=l[8])

In [47]:
l

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

In [48]:
nutz

Nutz(ma='DF', area=0.0, v_lh=300, v_nh=0, dring=2, behoerde=0, zeitp=3, schlaeg=4, rueck=35, text='El Kronen freistellen', error=False)

In [49]:
nutz.check_attr()

In [50]:
nutz

Nutz(ma='DF', area=0.0, v_lh=300, v_nh=0, dring=2, behoerde=0, zeitp=3, schlaeg=4, rueck=35, text='El Kronen freistellen', error=True)